To-do:
1. Текст был очищен только от одного мусорного элемента в качестве примера. Исслудйте данные через ноутбук или чере веб-интерфейс BigQuery на предмет других мусорных элементов в тексте, которые не несут в себе никакого особого смысла, а только создают шум в данных. Доработайте функцию очистки тектосвых данных, чтобы в нее можно было передать список ненужного мусора и разом выполнялась очистка
2. Проведите стратифицировнную кросс-валидуцию нейросетевого классификатора https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
3. Поэксперементируйте с гиперпараметрами нейросетевого классификатора, постарайтесь повысить качество его работы
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
4. Попробуйте использовать не Word2Vec для получения векторого представления текста, а TF-IDF преобразование http://zabaykin.ru/?p=558 http://nlpx.net/archives/57
5. Попробуйте использовать более тонко настриваемые алгоритмы нейросетей, например из этого видео https://www.youtube.com/watch?v=cPkH1k3U1c8 

In [210]:
import warnings
warnings.filterwarnings('ignore')

from google.oauth2 import service_account
import pandas_gbq 

import numpy as np
import pandas as pd
import math as mt
import datetime as dt

from langdetect import detect
import re

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix
from gensim.models.word2vec import Word2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt

In [211]:
CREDENTIALS = 

In [212]:
#funtion for getting fresh data from DWH for workload model
"""[summary]
Funtion for getting fresh data from BigQuery for workload scoring model
[description]
Credentials - google service account object with credentials data for project
[example]
Input: Credentials = credentials_object
Output: description	                                        channel	 category	category_flag
        \nChat transcript:\nVisitor: I want to buy wit...	chat	 ps	        1
        \nChat transcript:\nVisitor: hell i had a prob...	chat	 ps	        1
        \nChat transcript:\nVisitor: لا استطيع الشراء ...	 chat	  ps	     1
"""
def getDwhData(Credentials):
    statement_bigquery_sql = " ".join(["select description, channel, case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 'ps'",
                                       "else 'other'",
                                       "end as category,",
                                       "case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 0",
                                       "else 1",
                                       "end as category_flag",
                                       "from `xsolla_summer_school.customer_support`",
                                       "where manual_category is not null and",
                                       "manual_category <> '' and",
                                       "description is not null and",
                                       "description <> '' and",
                                       "channel is not null and",
                                       "channel <> '' and",
                                       "channel in ('chat','facebook')"])
    
    dataframe_bigquery = pandas_gbq.read_gbq(statement_bigquery_sql,project_id='findcsystem', credentials=Credentials, dialect='standard')

    return dataframe_bigquery


"""[summary]
Function for transform text to lower case
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["text_1","text_2"]
"""
def lowerCase(Corpus):
    corpus = [i.lower().replace('\n',' ') for i in Corpus]
    return corpus


"""[summary]
Function for getting language of text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["en","ru"]
"""
def getTextLanguage(Corpus):
    txt_lang = []
    for txt in Corpus:
        try:
            lang = detect(txt)
            txt_lang.append(lang)
        except:
            lang = 'error'
            txt_lang.append(lang)
    
    return txt_lang


"""[summary]
Function for tokenization text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["word1 word2","word3 word4"]
Output: [["word1","word2"],["word3","word4"]]
"""  
def textToTokens(Corpus):
    corpus = [i.split() for i in Corpus]
    return corpus 


"""[summary]
Function for clear text after garbage
[description]
Corpus - list or array object, with text data
listSubstr - list of strings, regular expressions
[example]
Input: Corpus = [["word1","word2"],["word3","word4"]]
       Substr = [r'word1]
Output: [["word2"],["word3","word4"]]
"""  
def clearTextAfterGarbage(Corpus,listSubstr):
    clear_corpus = []
    for text in Corpus:
        indexes = []
        text_len = len(text)
        try:
            for i in range(0,text_len):
                for Substr in listSubstr:
                    res = re.search(Substr,text[i])
                    if res != None:
                        indexes.append(i)
                
            #delete garbage word from text
            delta = 0
            indexes.sort()
            for index in indexes:
                del text[index - delta]
                delta += 1
            clear_corpus.append(text)
        except:
            clear_corpus.append("error")
        
    return clear_corpus


"""[summary]
Build word vector by using pre-trained Word2Vec model
[description]
Size - lenght of vector
Word2Vec_Model - gensim object
"""  
def buildWordVector(Text,Size,Word2Vec_Model):
    vec = np.zeros(Size).reshape((1,Size))
    count = 0.

    for word in Text:
        try:
            vec += Word2Vec_Model[word].reshape((1,Size))
            count += 1.
        except KeyError:
            continue
    
    if count != 0:
        vec /= count
    
    return vec

def mergeList(lst):
    all = []
    for l in lst:
        all = all + l
    return all    

RAWDATA 

In [4]:
#getting data from dwh
SupportRawDataframe = getDwhData(CREDENTIALS)
SupportRawDataframe.shape

Downloading: 100%|███████████████████████████████████████████████████████████| 23450/23450 [00:12<00:00, 1867.06rows/s]


(23450, 4)

In [5]:
SupportRawDataframe.head(10)

,description,channel,category,category_flag
0,\nChat transcript:\nVisitor: I want to buy wit...,chat,ps,0
1,\nChat transcript:\nVisitor: hell i had a prob...,chat,ps,0
2,\nChat transcript:\nVisitor: لا استطيع الشراء ...,chat,ps,0
3,\nChat transcript:\nVisitor: im having trouble...,chat,ps,0
4,\nChat transcript:\nVisitor: Hi\nAna: Hello. H...,chat,ps,0
5,\nChat transcript:\nVisitor: Здраствуйте\nAna:...,chat,ps,0
6,\nChat transcript:\nVisitor: hello\nVisitor: w...,chat,ps,0
7,\nChat transcript:\nVisitor: not letting me bu...,chat,ps,0
8,\nChat transcript:\nVisitor: مرحبا\nAna: Hello...,chat,ps,0
9,\nChat transcript:\nVisitor: hi im trying to b...,chat,ps,0


In [6]:
SupportRawDataframe.tail(10)

,description,channel,category,category_flag
23440,"\nChat transcript:\nVisitor: Hello, I bought p...",chat,other,1
23441,\nChat transcript:\nVisitor: Good day\n\nI acc...,facebook,other,1
23442,\nChat transcript:\nVisitor: payment not going...,chat,other,1
23443,\nChat transcript:\nVisitor: Paid for faceit s...,chat,other,1
23444,"\nChat transcript:\nVisitor: Hello, I am tryin...",chat,other,1
23445,"\nChat transcript:\nVisitor: Hi, i made a pruc...",chat,other,1
23446,"\nChat transcript:\nVisitor: Hi, how long will...",chat,other,1
23447,\nChat transcript:\nVisitor: I bought playerun...,chat,other,1
23448,\nChat transcript:\nVisitor: Good day i took t...,chat,other,1
23449,\nChat transcript:\nVisitor: hi\nVisitor: hell...,chat,other,1


In [81]:
SupportRawDataframe2 = SupportRawDataframe[0:2000]
SupportRawDataframe2

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
...,...,...,...,...,...
1995,chat transcript:visitor: i try again it dosent...,chat,ps,0,en
1996,chat transcript:visitor: не проходит платеж 25...,chat,ps,0,ru
1997,chat transcript:visitor: hi. i cant pay with m...,chat,ps,0,en
1998,chat transcript:visitor: i try purchase some c...,chat,ps,0,en


DATA PREPROC

In [213]:
#transform text to lower case
corpus = SupportRawDataframe.description
corpus.astype('str')

corpus = lowerCase(corpus)

#getting language for text corpus
corpus_lang = getTextLanguage(corpus)

In [ ]:
corpus_lang

In [214]:
#new dataframe with texts in lower case, without /n symbol and with lang for text
SupportRawDataframe['description'] = corpus
SupportRawDataframe['lang'] = corpus_lang

In [25]:
SupportRawDataframe2.head(10)

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
5,chat transcript:visitor: здраствуйтеana: здрав...,chat,ps,0,ru
6,chat transcript:visitor: hellovisitor: why i c...,chat,ps,0,en
7,chat transcript:visitor: not letting me buy an...,chat,ps,0,en
8,chat transcript:visitor: مرحباana: hello!visit...,chat,ps,0,en
9,chat transcript:visitor: hi im trying to buy v...,chat,ps,0,en


In [ ]:
SupportRawDataframe2.tail(10)

In [215]:
#getting only en texts
SupportDataframe_eng = SupportRawDataframe[SupportRawDataframe.lang == 'en'][:]
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
...,...,...,...,...,...
23445,"chat transcript:visitor: hi, i made a pruchase...",chat,other,1,en
23446,"chat transcript:visitor: hi, how long will it ...",chat,other,1,en
23447,chat transcript:visitor: i bought playerunknow...,chat,other,1,en
23448,chat transcript:visitor: good day i took the w...,chat,other,1,en


In [216]:
#text tekenization
tokenization = textToTokens(SupportDataframe_eng.description)
SupportDataframe_eng['description'] = tokenization

In [150]:
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"[chat, transcript:visitor:, i, want, to, buy, ...",chat,ps,0,en
1,"[chat, transcript:visitor:, hell, i, had, a, p...",chat,ps,0,en
2,"[chat, transcript:visitor:, لا, استطيع, الشراء...",chat,ps,0,en
3,"[chat, transcript:visitor:, im, having, troubl...",chat,ps,0,en
4,"[chat, transcript:visitor:, hiana:, hello., ho...",chat,ps,0,en
...,...,...,...,...,...
1990,"[chat, transcript:visitor:, how, get, nx, visi...",chat,ps,0,en
1991,"[chat, transcript:visitor:, why, i, cant, buy,...",chat,ps,0,en
1995,"[chat, transcript:visitor:, i, try, again, it,...",chat,ps,0,en
1997,"[chat, transcript:visitor:, hi., i, cant, pay,...",chat,ps,0,en


In [217]:
texts = list(SupportDataframe_eng.description)

In [218]:
#cleaning text after garbage
tests_clear = clearTextAfterGarbage(texts, [r'chat', r'transcript', r'hi', r'hello'])

In [170]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mural\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [173]:
nltk_stop_words = nltk.corpus.stopwords.words('english')

Можно воспользоваться этим списком для удаления мусора, но мой ноутбук вычисляет слишком долго, чтоб добавлять эти слова в список мусорных.

In [179]:
def createGarbageList(own_list=None):
    if own_list is None:
        own_list = []
    for word in nltk_stop_words:
        re_word = r''+word
        own_list.append(re_word) 
    return own_list    

In [180]:
#garbageList = createGarbageList([r'chat', r'transcript'])

In [185]:
#tests_clear = clearTextAfterGarbage(texts, garbageList)

In [219]:
SupportDataframe_eng['description'] = tests_clear
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"[i, want, to, buy, with, paypal, and, show, me...",chat,ps,0,en
1,"[hell, i, had, a, problem, yesterday, about, s...",chat,ps,0,en
2,"[لا, استطيع, الشراء, ومعلومات, الشراء, صحيحة, ...",chat,ps,0,en
3,"[im, having, trouble, with, the, new, xsolla, ...",chat,ps,0,en
4,"[hiana:, hello., how, can, i, help, you?visito...",chat,ps,0,en
...,...,...,...,...,...
23445,"[hi,, i, made, a, pruchase, of, $25, dollars, ...",chat,other,1,en
23446,"[hi,, how, long, will, it, take, my, friend, t...",chat,other,1,en
23447,"[i, bought, playerunknowns, battlegrounds, and...",chat,other,1,en
23448,"[good, day, i, took, the, wrong, package, i, w...",chat,other,1,en


TRAIN TEST SPLIT DATAFRAMES

In [220]:
#list of unique categories
unique_categories = np.unique(SupportDataframe_eng.category)
unique_categories

array(['other', 'ps'], dtype=object)

In [221]:
descriptions = SupportDataframe_eng['description']
categories = SupportDataframe_eng['category_flag']
XTrain,XTest,YTrain,YTest = train_test_split(descriptions,
                                             categories,
                                             stratify = categories,
                                             test_size = 0.2,
                                             random_state = 40)

FROM TEXTS TO VECTORS

In [251]:
#initialize Word2Vec model for embedding words to vectors
NDim = 100
Imdb_w2v = Word2Vec(size = NDim,min_count = 10)
Imdb_w2v.build_vocab(descriptions)

In [252]:
Imdb_w2v.train(XTrain,total_examples = Imdb_w2v.corpus_count,epochs = Imdb_w2v.epochs)

(7761368, 11566275)

In [253]:
#embedding training messages to vectors for neutral classifier
TrainVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTrain])

In [ ]:
TrainVecs

In [254]:
Imdb_w2v.train(XTest, total_examples = Imdb_w2v.corpus_count, epochs = Imdb_w2v.epochs)

(1883639, 2809200)

In [255]:
TestVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTest])

In [ ]:
TestVecs

CROSSVALIDATION AND BUILD CLASSIFIER

In [256]:
TextClassifier = MLPClassifier(hidden_layer_sizes = (20,10), max_iter = 1000, random_state = 40)

In [257]:
skf = StratifiedKFold(n_splits=5)

In [258]:
Scores = cross_val_score(TextClassifier, TrainVecs, YTrain, cv = skf)
print(Scores)
print(np.mean(Scores))

[0.89344262 0.89383294 0.90320062 0.8992581  0.90238188]
0.8984232349714583


In [259]:
TextClassifier.fit(TrainVecs,YTrain)

MLPClassifier(hidden_layer_sizes=(20, 10), max_iter=1000, random_state=40)

In [260]:
pred = TextClassifier.predict(TestVecs)
print(confusion_matrix(YTest,pred))
print(classification_report(YTest,pred))

[[ 505  131]
 [ 457 2109]]
              precision    recall  f1-score   support

           0       0.52      0.79      0.63       636
           1       0.94      0.82      0.88      2566

    accuracy                           0.82      3202
   macro avg       0.73      0.81      0.75      3202
weighted avg       0.86      0.82      0.83      3202



TF-IDF

In [261]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [267]:
SupportDataframe_eng = SupportRawDataframe[SupportRawDataframe.lang == 'en'][:]
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
...,...,...,...,...,...
23445,"chat transcript:visitor: hi, i made a pruchase...",chat,other,1,en
23446,"chat transcript:visitor: hi, how long will it ...",chat,other,1,en
23447,chat transcript:visitor: i bought playerunknow...,chat,other,1,en
23448,chat transcript:visitor: good day i took the w...,chat,other,1,en


In [268]:
description_texts = SupportDataframe_eng['description']

In [269]:
categories = SupportDataframe_eng['category_flag']

In [277]:
XTrain,XTest,YTrain,YTest = train_test_split(description_texts,
                                             categories,
                                             stratify = categories,
                                             test_size = 0.2,
                                             random_state = 40)

In [278]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', RandomForestClassifier())])

In [279]:
text_clf.fit(XTrain, YTrain)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier())])

In [280]:
predictions = text_clf.predict(XTest)
print(confusion_matrix(YTest,predictions))
print(classification_report(YTest,predictions))

[[ 290  346]
 [  44 2522]]
              precision    recall  f1-score   support

           0       0.87      0.46      0.60       636
           1       0.88      0.98      0.93      2566

    accuracy                           0.88      3202
   macro avg       0.87      0.72      0.76      3202
weighted avg       0.88      0.88      0.86      3202

